# Django Configuration and setup #
I used the Anaconda base environment and installed Django creating a smartnotes project with ` django-admi startproject smartnotes ` n , an app within (webpage) called home and the functions of that app in the views.py file which is created with the ` django-admin startapp [app] `

To start the Django server for testing use ` python manage.py runserver `

## The path ##
<img src=assets/ThePath.jpg>

<img src=assets/MVTFramework.jpg>

The Model View Template.  The model layer handles the data and how it's stored.  The view is where we employ the functions to work with the data and the template is how it will render in HTML.  Django Template Language is what the template employs as code in the view layer is passed down to the template layer and that's what makes the html in a manner similar to a JavaScript insertion.  DTL allows for rich HTML formatting without a lot of HTML coding. 

## Migrations ##
To keep databases in sync changes are maintained in a `migrations` folder and a `python manage.py migrate` command will synchronize these in the database. 

<img src=assets/Migrations.jpg>

# The Django Admin Page #

` /admin ` at the localhost URL for the python server will take you to the Django Admin page.

First thing is to create a super user with the `python manage.py createsuperuser` command. 

With the user you can use the built in admin page to configure users and group permissions to create and delete accounts, contents or modify privileges.  The `@login_required(login_url='/admin')` appearing in the views.py file will prevent following functions from running unless logged in with correct privilege and will redirect to the login URL supplied.  One line to secure code with a robust authentication system including encrypted passwords. 

# Django Object Relational Mapping System #

Class models get written then migrated into database tables where the class defines the table and the class attributes are columns in the database table. 

<img src=assets/ClassMapping.jpg>

This system is particularly useful for those who wish to use Python to script web applications using SQL databases.

For instance to create the database for a note taking app we'd want a database called *Notes* and the code that makes lives in the Django app folders' `models.py` file as: 
```
class Notes(models.Model):
    title = models.CharField(max_length=200)
    text = models.TextField()
    created = models.DateTimeField(auto_now_add=True)

```

And then a run of the `python manage.py makemigrations` would convert that to a script that upon next migration will create database table.  Then do `python manage.py migrate

When an app is initiated with django-admin startapp a folder is created and populated with other folders and files including an `admin.py` folder in which you must indicate where the file of models you wish to use are.  The models.py folder has the classes you wish to use in your app. `

## Django Shell ##

You can use this tool to check the contents of a database `python manage.py shell`  it constructs a command line interface with an awareness of the project and database so you can directly query and even add to the database from the command line. 
`Notes.objects.all()` will return a list of all the objects in the database called Notes.  Other methods in the database objects available will filter including or excluding content. 